In [29]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [30]:
# Read in data
supply_data = pd.read_csv('https://raw.githubusercontent.com/ZorroHZR/MMAI-5000/main/Omis%206000%20Final/ecogreen_energy_supply.csv')
demand_data = pd.read_csv('https://raw.githubusercontent.com/ZorroHZR/MMAI-5000/main/Omis%206000%20Final/ecogreen_energy_demand.csv')

In [31]:
# Extract relevant data
fixed_costs = supply_data['Fixed'].tolist()
capacities = supply_data['Capacity'].tolist()
variable_costs = supply_data.iloc[:, 3:].values.tolist()
demands = demand_data['Demand'].tolist()

# Create model
m = gp.Model("EcoGreen_Energy")

# Add variables
open_vars = m.addVars(20, vtype=GRB.BINARY, name="open")
supply_vars = m.addVars(20, 10, name="supply")

# Set objective
m.setObjective(gp.quicksum(open_vars[i] * fixed_costs[i] for i in range(20)) + 
               gp.quicksum(supply_vars[i, j] * variable_costs[i][j] for i in range(20) for j in range(10)),
               GRB.MINIMIZE)

# Add constraints
# Capacity constraints
m.addConstrs((gp.quicksum(supply_vars[i, j] for j in range(10)) <= capacities[i] * open_vars[i] for i in range(20)), name="Capacity")

# Demand constraints
m.addConstrs((gp.quicksum(supply_vars[i, j] for i in range(20)) == demands[j] for j in range(10)), name="Demand")

# Logical constraints
m.addConstr(open_vars[9] + open_vars[14] + open_vars[19] <= 1, name="Logical1")
m.addConstr(open_vars[2] <= open_vars[3], name="Logical2")
m.addConstr(open_vars[2] <= open_vars[4], name="Logical3")
m.addConstr(open_vars[4] <= open_vars[7] + open_vars[8], name="Logical4")
m.addConstr(gp.quicksum(open_vars[i] for i in range(10)) <= 2 * gp.quicksum(open_vars[i] for i in range(10, 20)), name="Logical5")

# Energy mix constraints
m.addConstr(gp.quicksum(gp.quicksum(supply_vars[i,j] for j in range(10)) for i in range(5)) >= 
            0.3 * gp.quicksum(gp.quicksum(supply_vars[i,j] for j in range(10)) for i in range(20)), name="EnergyMix")

# No single plant provides >50% of a province's needs
m.addConstrs((supply_vars[i,j] <= 0.5 * demands[j] for i in range(20) for j in range(10)), name="MaxSupply")

# Optimize model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 236 rows, 220 columns and 850 nonzeros
Model fingerprint: 0x72ed505d
Variable types: 200 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+05]
  Objective range  [2e-01, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
Found heuristic solution: objective 3.232874e+08
Presolve removed 200 rows and 0 columns
Presolve time: 0.00s
Presolved: 36 rows, 220 columns, 500 nonzeros
Variable types: 200 continuous, 20 integer (20 binary)

Root relaxation: objective 1.685904e+08, 79 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Tim

In [32]:
print(f"Optimal objective value: ${m.objVal:.2f}")
print(f"Number of opened power plants: {sum([int(open_vars[i].x) for i in range(20)])}")

Optimal objective value: $193005688.79
Number of opened power plants: 11


In [33]:
# (g) Print the number of opened power plants
print(f"In the optimal solution, {sum([int(open_vars[i].x) for i in range(20)])} power plants will be established.")


In the optimal solution, 11 power plants will be established.


In [34]:
# (h) Print the highest and lowest number of power plants supplying a single province
max_plants = max([sum([1 for i in range(20) if supply_vars[i,j].x > 0]) for j in range(10)])
min_plants = min([sum([1 for i in range(20) if supply_vars[i,j].x > 0]) for j in range(10)])
print(f"The highest number of power plants supplying a single province is {max_plants}.")
print(f"The lowest number of power plants supplying a single province is {min_plants}.")


The highest number of power plants supplying a single province is 4.
The lowest number of power plants supplying a single province is 2.


In [36]:
# (k) Omit the 50% rule and print the number of opened power plants
max_supply_constrs = m.getConstrs()
for constr in max_supply_constrs:
    if constr.ConstrName == "MaxSupply":
        m.remove(constr)

m.optimize()
print(f"Without the 50% rule, {sum([int(open_vars[i].x) for i in range(20)])} power plants will be established.")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 236 rows, 220 columns and 850 nonzeros
Model fingerprint: 0x72ed505d
Variable types: 200 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+05]
  Objective range  [2e-01, 3e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
Presolved: 36 rows, 220 columns, 500 nonzeros

Continuing optimization...


Cutting planes:
  Cover: 1
  Implied bound: 1
  MIR: 2
  Flow cover: 1



Explored 1 nodes (161 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 12 (of 12 available processors)

Solution count 5: 1.93006e+08 1.97542e+08 2.00011e+08 ... 3.23287e+08

Optimal solution found (tolerance 1.00e-04)
Best objective 1.930056887899e+08, best bound 1.930050012520e+08, gap 0.0004%
Without the 50% rule, 11 power plants will be established.
